In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install -q transformers

In [16]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

**Load Data**

In [17]:
data = pd.read_csv("/content/drive/MyDrive/winemag-data_first150k.csv")
data

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


**Data Cleaning and  Imputing of null values**

In [18]:
#drop the index column
data=data.drop(columns=['Unnamed: 0'])
data["price"].describe()

count    137235.000000
mean         33.131482
std          36.322536
min           4.000000
25%          16.000000
50%          24.000000
75%          40.000000
max        2300.000000
Name: price, dtype: float64

In [19]:
data['description'].isna().sum()

0

In [20]:
# drop null and duplicate values
data.dropna(subset=['price', "points"], inplace=True)
data.drop_duplicates(subset=['description'], inplace=True)
data.nunique()

country           46
description    89108
designation    28345
points            21
price            357
province         446
region_1        1197
region_2          18
variety          619
winery         13852
dtype: int64

In [21]:
#replace nan values for text features with empty string
data[['country', 'designation', 'region_1', 'region_2', 'province', 'variety', 'winery']] = data[['country','designation', 'region_1', 'region_2', 'province', 'variety', 'winery']].fillna("")
data.isnull().sum()

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
variety        0
winery         0
dtype: int64

In [22]:
four_bins = pd.qcut(data["price"], q=4)
print(four_bins)

two_bins = pd.qcut(data["price"], q=2)
print(two_bins)

0         (40.0, 2300.0]
1         (40.0, 2300.0]
2         (40.0, 2300.0]
3         (40.0, 2300.0]
4         (40.0, 2300.0]
               ...      
149634     (3.999, 16.0]
149635     (3.999, 16.0]
149637     (3.999, 16.0]
149638     (3.999, 16.0]
149639     (3.999, 16.0]
Name: price, Length: 89108, dtype: category
Categories (4, interval[float64, right]): [(3.999, 16.0] < (16.0, 25.0] < (25.0, 40.0] <
                                           (40.0, 2300.0]]
0         (25.0, 2300.0]
1         (25.0, 2300.0]
2         (25.0, 2300.0]
3         (25.0, 2300.0]
4         (25.0, 2300.0]
               ...      
149634     (3.999, 25.0]
149635     (3.999, 25.0]
149637     (3.999, 25.0]
149638     (3.999, 25.0]
149639     (3.999, 25.0]
Name: price, Length: 89108, dtype: category
Categories (2, interval[float64, right]): [(3.999, 25.0] < (25.0, 2300.0]]


In [23]:
four_categories = four_bins.cat.categories.tolist()
four_target_names = [str(i) for i in four_categories]

two_categories = two_bins.cat.categories.tolist()
two_target_names = [str(i) for i in two_categories]

In [24]:
data["four_price_categorical"] = four_bins.cat.codes
data["two_price_categorical"] = two_bins.cat.codes
data

,country,description,designation,points,price,province,region_1,region_2,variety,winery,four_price_categorical,two_price_categorical
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,3,1
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez,3,1
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,3,1
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,3,1
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,,Provence red blend,Domaine de la Bégude,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
149634,France,Atypically light in body and reticent on the n...,,84,15.0,Alsace,Alsace,,Gewürztraminer,W. Gisselbrecht,0,0
149635,US,A Syrah-Grenache blend that's dry and rustical...,Bungalow Red,84,15.0,California,Santa Barbara County,Central Coast,Syrah-Grenache,Casa Barranca,0,0
149637,US,"Outside of the vineyard, wines like this are w...",,84,6.0,California,California,California Other,Merlot,Delicato,0,0
149638,Argentina,"Heavy and basic, with melon and pineapple arom...",,84,9.0,Mendoza Province,Uco Valley,,Sauvignon Blanc,Finca El Portillo,0,0


In [25]:
#Convert text features into sentences
data['country'] = 'The country of this wine is ' + data['country'] + '. '
data['designation'] = 'The designation of this wine is ' + data['designation'] + '. '
data['province'] = 'The province of this wine is ' + data['province'] + '. '
data['region_1'] = 'The region of this wine is ' + data['region_1'] + '. '
data['region_2'] = 'The secondary region of this wine is ' + data['region_2'] + '. '
data['variety'] = 'The variety of this wine is ' + data['variety'] + '. '
data['winery'] = 'The winery from this wine is ' + data['winery'] + '. '
data['points'] = 'The rating given to this wine is ' + data["points"].astype("string") + ". "

In [26]:
data['description'] = data['description'].astype(str)
data['description_noDigits'] = data['description'].str.replace(r'\d', '', regex=True)

In [27]:
data["description_features"] = data["description_noDigits"] + " " + data["points"].astype(str) + data['variety'] + data['country'] + data['province'] + data["region_1"] + data['winery'] + data["region_2"] + data["designation"]
data["description_features"].iloc[0]

"This tremendous % varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy –. The rating given to this wine is 96. The variety of this wine is Cabernet Sauvignon. The country of this wine is US. The province of this wine is California. The region of this wine is Napa Valley. The winery from this wine is Heitz. The secondary region of this wine is Napa. The designation of this wine is Martha's Vineyard. "

**10,000 sample predictions**

In [28]:
data_selection = 10000
four_sample_data = data.sample(data_selection, random_state=1)
two_sample_data = data.sample(data_selection, random_state=1)
# (train_texts, test_texts, train_labels, test_labels) = train_test_split(list(data["description_features"][:data_selection]), list(df["price_categorical"][:data_selection]), test_size=0.2, stratify=list(df["price_categorical"][:data_selection]), random_state=1)

(four_train_data, four_test_data) = train_test_split(four_sample_data, test_size=0.2, random_state=1)
(two_train_data, two_test_data) = train_test_split(two_sample_data, test_size=0.2, random_state=1)

In [29]:
# (train_texts, valid_texts, train_labels, valid_labels) = train_test_split(train_texts, train_labels, test_size=0.25, stratify=train_labels, random_state=1)
# type(train_texts)

(four_train_data, four_valid_data) = train_test_split(four_train_data, test_size=0.25, random_state=1)
(two_train_data, two_valid_data) = train_test_split(two_train_data, test_size=0.25, random_state=1)


In [30]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [31]:
four_percs = pd.qcut(four_train_data["description_features"].str.len(), q=10)
print(four_percs)

two_percs = pd.qcut(two_train_data["description_features"].str.len(), q=10)
print(two_percs)

49592       (545.0, 561.0]
50783       (481.9, 508.0]
149343      (597.0, 623.0]
59602      (661.0, 1022.0]
111711      (481.9, 508.0]
                ...       
81854       (508.0, 528.0]
56561       (578.0, 597.0]
56273       (481.9, 508.0]
37331     (329.999, 481.9]
105197      (545.0, 561.0]
Name: description_features, Length: 6000, dtype: category
Categories (10, interval[float64, right]): [(329.999, 481.9] < (481.9, 508.0] < (508.0, 528.0] <
                                            (528.0, 545.0] ... (578.0, 597.0] < (597.0, 623.0] <
                                            (623.0, 661.0] < (661.0, 1022.0]]
49592       (545.0, 561.0]
50783       (481.9, 508.0]
149343      (597.0, 623.0]
59602      (661.0, 1022.0]
111711      (481.9, 508.0]
                ...       
81854       (508.0, 528.0]
56561       (578.0, 597.0]
56273       (481.9, 508.0]
37331     (329.999, 481.9]
105197      (545.0, 561.0]
Name: description_features, Length: 6000, dtype: category
Categories (10, in

In [32]:
four_max_length = 460
two_max_length = 460

In [33]:
def create_bert_model(bert_model,
                      experiment,
                      num_classes=4,
                      hidden_size = 256,
                      dropout=0.3,
                      learning_rate=1e-5):

    bert_model.trainable = True

    if experiment == "two class":
      input_ids = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='input_ids_layer')
      token_type_ids = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='token_type_ids_layer')
      attention_mask = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='attention_mask_layer')
    else:
      input_ids = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='input_ids_layer')
      token_type_ids = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='token_type_ids_layer')
      attention_mask = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]
    dropout_layer = tf.keras.layers.Dropout(dropout)(cls_token)
    hidden_layer = tf.keras.layers.Dense(hidden_size, activation='swish', name='hidden_layer')(dropout_layer)
    hidden_layer_2 = tf.keras.layers.Dense(hidden_size/2, activation='swish', name='hidden_layer_2')(hidden_layer)

    if experiment == "two class":
      classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden_layer_2)
      classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
      classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss = 'binary_crossentropy',
              metrics = 'accuracy')
    else:
      classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden_layer_2)
      classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
      classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')

    return classification_model

In [34]:
four_cls_bert_model = create_bert_model(bert_model, experiment="four class")

four_nptrain_labels = np.asarray(four_train_data['four_price_categorical'])
four_npvalid_labels = np.asarray(four_valid_data['four_price_categorical'])
four_nptest_labels = np.asarray(four_test_data['four_price_categorical'])

train_encodings = bert_tokenizer(list(four_train_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(four_valid_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')
four_test_encodings = bert_tokenizer(list(four_test_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')

four_cls_bert_model_history_10k = four_cls_bert_model.fit(x=[train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                              y=four_nptrain_labels,
                                              validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                              four_npvalid_labels),
                                              shuffle=True,
                                              batch_size=8,
                                              validation_batch_size=8,
                                              epochs=4)

predictions = four_cls_bert_model.predict([four_test_encodings.input_ids, four_test_encodings.token_type_ids, four_test_encodings.attention_mask])
predictions = tf.argmax(predictions, axis=-1)
print(classification_report(four_nptest_labels, predictions.numpy(), target_names=four_target_names))

four_test_score = four_cls_bert_model.evaluate([four_test_encodings.input_ids, four_test_encodings.token_type_ids, four_test_encodings.attention_mask],
                                                  four_nptest_labels)

print('Four-class 10k test loss:', four_test_score[0])
print('Four-class 10k test accuracy:', four_test_score[1])

Epoch 1/4


750/750 [==============================] - 784s 986ms/step - loss: 1.2165 - accuracy: 0.4272 - val_loss: 1.0956 - val_accuracy: 0.4985
Epoch 2/4
750/750 [==============================] - 743s 991ms/step - loss: 1.0001 - accuracy: 0.5462 - val_loss: 0.9657 - val_accuracy: 0.5665
Epoch 3/4
750/750 [==============================] - 743s 991ms/step - loss: 0.8911 - accuracy: 0.6008 - val_loss: 0.9523 - val_accuracy: 0.5695
Epoch 4/4
63/63 [==============================] - 84s 1s/step
                precision    recall  f1-score   support

 (3.999, 16.0]       0.72      0.66      0.69       569
  (16.0, 25.0]       0.42      0.42      0.42       511
  (25.0, 40.0]       0.42      0.50      0.46       426
(40.0, 2300.0]       0.74      0.70      0.72       494

      accuracy                           0.57      2000
     macro avg       0.58      0.57      0.57      2000
  weighted avg       0.59      0.57      0.58      2000

63/63 [==============================] - 85s 1s/step - loss: 

In [35]:
two_cls_bert_model = create_bert_model(bert_model, experiment="two class")

two_nptrain_labels = np.asarray(two_train_data['two_price_categorical'])
two_npvalid_labels = np.asarray(two_valid_data['two_price_categorical'])
two_nptest_labels = np.asarray(two_test_data['two_price_categorical'])

train_encodings = bert_tokenizer(list(two_train_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(two_valid_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')
two_test_encodings = bert_tokenizer(list(two_test_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')

two_cls_bert_model_history_10k = two_cls_bert_model.fit(x=[train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                              y=two_nptrain_labels,
                                              validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                              two_npvalid_labels),
                                              shuffle=True,
                                              batch_size=8,
                                              validation_batch_size=8,
                                              epochs=4)

predictions = two_cls_bert_model.predict([two_test_encodings.input_ids, two_test_encodings.token_type_ids, two_test_encodings.attention_mask])
predictions = tf.argmax(predictions, axis=-1)
print(classification_report(two_nptest_labels, predictions.numpy(), target_names=two_target_names))

two_test_score = two_cls_bert_model.evaluate([two_test_encodings.input_ids, two_test_encodings.token_type_ids, two_test_encodings.attention_mask],
                                                  two_nptest_labels)

print('Two-class 10k test loss:', two_test_score[0])
print('Two-class 10k test accuracy:', two_test_score[1])

Epoch 1/4


750/750 [==============================] - 789s 997ms/step - loss: 0.2764 - accuracy: 0.8822 - val_loss: 0.4850 - val_accuracy: 0.7890
Epoch 2/4
750/750 [==============================] - 745s 993ms/step - loss: 0.1764 - accuracy: 0.9300 - val_loss: 0.6343 - val_accuracy: 0.7945
Epoch 3/4
750/750 [==============================] - 743s 991ms/step - loss: 0.1098 - accuracy: 0.9592 - val_loss: 0.7546 - val_accuracy: 0.7675
Epoch 4/4
63/63 [==============================] - 84s 1s/step
                precision    recall  f1-score   support

 (3.999, 25.0]       0.54      1.00      0.70      1080
(25.0, 2300.0]       0.00      0.00      0.00       920

      accuracy                           0.54      2000
     macro avg       0.27      0.50      0.35      2000
  weighted avg       0.29      0.54      0.38      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 85s 1s/step - loss: 0.7796 - accuracy: 0.8135
Two-class 10k test loss: 0.779553234577179
Two-class 10k test accuracy: 0.8134999871253967


**20,000 sample predictions**

In [36]:
data_selection = 20000
four_sample_data = data.sample(data_selection, random_state=1)
two_sample_data = data.sample(data_selection, random_state=1)
# (train_texts, test_texts, train_labels, test_labels) = train_test_split(list(data["description_features"][:data_selection]), list(df["price_categorical"][:data_selection]), test_size=0.2, stratify=list(df["price_categorical"][:data_selection]), random_state=1)

(four_train_data, four_test_data) = train_test_split(four_sample_data, test_size=0.2, random_state=1)
(two_train_data, two_test_data) = train_test_split(two_sample_data, test_size=0.2, random_state=1)

In [37]:
# (train_texts, valid_texts, train_labels, valid_labels) = train_test_split(train_texts, train_labels, test_size=0.25, stratify=train_labels, random_state=1)
# type(train_texts)

(four_train_data, four_valid_data) = train_test_split(four_train_data, test_size=0.25, random_state=1)
(two_train_data, two_valid_data) = train_test_split(two_train_data, test_size=0.25, random_state=1)


In [38]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [39]:
four_percs = pd.qcut(four_train_data["description_features"].str.len(), q=10)
print(four_percs)

two_percs = pd.qcut(two_train_data["description_features"].str.len(), q=10)
print(two_percs)

143931      (481.0, 509.0]
88758       (598.0, 623.0]
100386      (623.0, 660.0]
6128      (355.999, 481.0]
31926     (355.999, 481.0]
                ...       
70132      (660.0, 1044.0]
9659        (529.0, 546.0]
111723    (355.999, 481.0]
78773       (546.0, 563.0]
98078       (563.0, 580.0]
Name: description_features, Length: 12000, dtype: category
Categories (10, interval[float64, right]): [(355.999, 481.0] < (481.0, 509.0] < (509.0, 529.0] <
                                            (529.0, 546.0] ... (580.0, 598.0] < (598.0, 623.0] <
                                            (623.0, 660.0] < (660.0, 1044.0]]
143931      (481.0, 509.0]
88758       (598.0, 623.0]
100386      (623.0, 660.0]
6128      (355.999, 481.0]
31926     (355.999, 481.0]
                ...       
70132      (660.0, 1044.0]
9659        (529.0, 546.0]
111723    (355.999, 481.0]
78773       (546.0, 563.0]
98078       (563.0, 580.0]
Name: description_features, Length: 12000, dtype: category
Categories (10, 

In [40]:
four_max_length = 460
two_max_length = 460

In [41]:
def create_bert_model(bert_model,
                      experiment,
                      num_classes=4,
                      hidden_size = 256,
                      dropout=0.3,
                      learning_rate=1e-5):

    bert_model.trainable = True

    if experiment == "two class":
      input_ids = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='input_ids_layer')
      token_type_ids = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='token_type_ids_layer')
      attention_mask = tf.keras.layers.Input(shape=(two_max_length,), dtype=tf.int64, name='attention_mask_layer')
    else:
      input_ids = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='input_ids_layer')
      token_type_ids = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='token_type_ids_layer')
      attention_mask = tf.keras.layers.Input(shape=(four_max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]
    dropout_layer = tf.keras.layers.Dropout(dropout)(cls_token)
    hidden_layer = tf.keras.layers.Dense(hidden_size, activation='swish', name='hidden_layer')(dropout_layer)
    hidden_layer_2 = tf.keras.layers.Dense(hidden_size/2, activation='swish', name='hidden_layer_2')(hidden_layer)

    if experiment == "two class":
      classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden_layer_2)
      classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
      classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss = 'binary_crossentropy',
              metrics = 'accuracy')
    else:
      classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden_layer_2)
      classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
      classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')

    return classification_model

In [42]:
four_cls_bert_model = create_bert_model(bert_model, experiment="four class")

four_nptrain_labels = np.asarray(four_train_data['four_price_categorical'])
four_npvalid_labels = np.asarray(four_valid_data['four_price_categorical'])
four_nptest_labels = np.asarray(four_test_data['four_price_categorical'])

train_encodings = bert_tokenizer(list(four_train_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(four_valid_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')
four_test_encodings = bert_tokenizer(list(four_test_data['description_features']), truncation=True, padding="max_length", max_length=four_max_length, return_tensors='tf')

four_cls_bert_model_history_20k = four_cls_bert_model.fit(x=[train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                              y=four_nptrain_labels,
                                              validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                              four_npvalid_labels),
                                              shuffle=True,
                                              batch_size=8,
                                              validation_batch_size=8,
                                              epochs=4)

predictions = four_cls_bert_model.predict([four_test_encodings.input_ids, four_test_encodings.token_type_ids, four_test_encodings.attention_mask])
predictions = tf.argmax(predictions, axis=-1)
print(classification_report(four_nptest_labels, predictions.numpy(), target_names=four_target_names))

four_test_score = four_cls_bert_model.evaluate([four_test_encodings.input_ids, four_test_encodings.token_type_ids, four_test_encodings.attention_mask],
                                                  four_nptest_labels)

print('Four-class 20k test loss:', four_test_score[0])
print('Four-class 20k test accuracy:', four_test_score[1])

Epoch 1/4


1500/1500 [==============================] - 1573s 1s/step - loss: 1.1169 - accuracy: 0.4737 - val_loss: 0.9939 - val_accuracy: 0.5452
Epoch 2/4
1500/1500 [==============================] - 1528s 1s/step - loss: 0.9420 - accuracy: 0.5773 - val_loss: 0.9020 - val_accuracy: 0.5990
Epoch 3/4
1500/1500 [==============================] - 1486s 991ms/step - loss: 0.8284 - accuracy: 0.6352 - val_loss: 0.9149 - val_accuracy: 0.5910
Epoch 4/4
125/125 [==============================] - 159s 1s/step
                precision    recall  f1-score   support

 (3.999, 16.0]       0.79      0.68      0.73      1104
  (16.0, 25.0]       0.47      0.50      0.49      1020
  (25.0, 40.0]       0.46      0.35      0.40       905
(40.0, 2300.0]       0.63      0.84      0.72       971

      accuracy                           0.60      4000
     macro avg       0.59      0.59      0.58      4000
  weighted avg       0.60      0.60      0.59      4000

125/125 [==============================] - 166s 1s/step

In [ ]:
two_cls_bert_model = create_bert_model(bert_model, experiment="two class")

two_nptrain_labels = np.asarray(two_train_data['two_price_categorical'])
two_npvalid_labels = np.asarray(two_valid_data['two_price_categorical'])
two_nptest_labels = np.asarray(two_test_data['two_price_categorical'])

train_encodings = bert_tokenizer(list(two_train_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(two_valid_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')
two_test_encodings = bert_tokenizer(list(two_test_data['description_features']), truncation=True, padding="max_length", max_length=two_max_length, return_tensors='tf')

two_cls_bert_model_history_20k = two_cls_bert_model.fit(x=[train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                              y=two_nptrain_labels,
                                              validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                              two_npvalid_labels),
                                              shuffle=True,
                                              batch_size=8,
                                              validation_batch_size=8,
                                              epochs=4)

predictions = two_cls_bert_model.predict([two_test_encodings.input_ids, two_test_encodings.token_type_ids, two_test_encodings.attention_mask])
predictions = tf.argmax(predictions, axis=-1)
print(classification_report(two_nptest_labels, predictions.numpy(), target_names=two_target_names))

two_test_score = two_cls_bert_model.evaluate([two_test_encodings.input_ids, two_test_encodings.token_type_ids, two_test_encodings.attention_mask],
                                                  two_nptest_labels)

print('Two-class 20k test loss:', two_test_score[0])
print('Two-class 20k test accuracy:', two_test_score[1])

Epoch 1/4


1500/1500 [==============================] - 1533s 993ms/step - loss: 0.2405 - accuracy: 0.8957 - val_loss: 0.4169 - val_accuracy: 0.8357
Epoch 2/4
  35/1500 [..............................] - ETA: 21:33 - loss: 0.1236 - accuracy: 0.9536

In [ ]:
# Valid accuracy comparison between 10k and 20k
plt.plot(four_cls_bert_model_history_10k.history['val_accuracy'])
plt.plot(four_cls_bert_model_history_20k.history['val_accuracy'])
plt.title('Model Valid Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['10k', '20k'], loc='upper left')
plt.show()

# Valid loss comparison between 10k and 20k
plt.plot(four_cls_bert_model_history_10k.history['val_loss'])
plt.plot(four_cls_bert_model_history_20k.history['val_loss'])
plt.title('Model Valid Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['10k', '20k'], loc='upper left')
plt.show()